# QRT Challenge Data 2024
## Data Specs (Copied from Official)
- `X_train` and `X_test` both have $35$ columns that represent the same explanatory variables but over different time periods. 

- `X_train` and `Y_train` share the same column `ID` - each row corresponds to a unique ID associated wwith a day and a country. 

- The target of this challenge `TARGET` in `Y_train` corresponds to the price change for daily futures contracts of 24H electricity baseload. 

- **You will notice some columns have missing values**.

Input data sets comprise 35 columns:

ID: Unique row identifier, associated with a day (DAY_ID) and a country (COUNTRY),

DAY_ID: Day identifier - dates have been anonymized, but all data corresponding to a specific day is consistent,

COUNTRY: Country identifier - DE = Germany, FR = France,
and then contains daily commodity price variations,

GAS_RET: European gas,

COAL_RET: European coal,

CARBON_RET: Carbon emissions futures,

#### Weather measures (daily, in the country x)

x_TEMP: Temperature,

x_RAIN: Rainfall,

x_WIND: Wind,


#### Energy production measures (daily, in the country x)

x_GAS: Natural gas,

x_COAL: Hard coal,

x_HYDRO: Hydro reservoir,

x_NUCLEAR: Daily nuclear production,

x_SOLAR: Photovoltaic,

x_WINDPOW: Wind power,

x_LIGNITE: Lignite,

#### Electricity use metrics (daily, in the country x)

x_CONSUMPTON: Total electricity consumption,

x_RESIDUAL_LOAD: Electricity consumption after using all renewable energies,

x_NET_IMPORT: Imported electricity from Europe,

x_NET_EXPORT: Exported electricity to Europe,

DE_FR_EXCHANGE: Total daily electricity exchange between Germany and France,

FR_DE_EXCHANGE: Total daily electricity exchange between France and Germany.

Output data sets are composed of two columns:

ID: Unique row identifier - corresponding to the input identifiers,

TARGET: Daily price variation for futures of 24H electricity baseload.


In [61]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import itertools as it
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import mutual_info_regression as mir
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder as onehot
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
import xgboost as xgb
import lightgbm as lgb
from sklearn.impute import KNNImputer

In [62]:
df = pd.read_csv('data/X_train_NHkHMNU.csv')
y = pd.read_csv('data/y_train_ZAN5mwg.csv')
true_test = pd.read_csv('data/X_test_final.csv')
pd.set_option('display.max_columns', None)


In [56]:
# for fitting two part linear regression to WIND_SQCB / WINDPOW to determine excess production
# in general, given two series and a threshold (boolean) function, the fn will split the the x and y values
# based on the function and then do ols on each side, returning a new series with the residual
# of each y-value from the fitted line

def lr_sd(lr, x, y):
    return (y - lr.predict(x)).pow(2).sum() / y.size

class SDLinReg:
    def __init__(self):
        return None
    
    def fit(self, d, x, y, f=None):
        self.x = x
        self.y = y
        data = d[[x, y]].copy()
        if f is None:
            d1 = data
            lr1 = LinearRegression()
            lr1.fit(d1[x].values.reshape(-1, 1), d1[y])
            sd1 = lr_sd(lr1, d1[[x]], d1[y])
            self.p = lambda r : (r[1] - lr1.predict([[r[0]]])) / sd1
        else:
            d1, d2 = data[f(data[x], data[y])], data[~ f(data[x], data[y])]
            lr1, lr2 = LinearRegression(), LinearRegression()
            lr1.fit(d1[x].values.reshape(-1, 1), d1[y])
            lr2.fit(d2[x].values.reshape(-1, 1), d2[y])
            sd1, sd2 = lr_sd(lr1, d1[[x]], d1[y]), lr_sd(lr2, d2[[x]], d2[y])
            self.p = lambda r : ((r[1] - lr1.predict([[r[0]]])) / sd1) if f(r[0], r[1]) else ((r[1] - lr2.predict([[r[0]]])) / sd2)
        return

    def predict(self, d, x=None, y=None, debug=False):
        if x is None:
            x = self.x
        if y is None:
            y = self.y
        data = d[[x, y]]
        if debug:
            i = 0
            for row in data.itertuples(index=False):
                print("row:", row)
                print("p(row):", self.p(row))
                print("float:", float(self.p(row)))
                i += 1
                if i > 20:
                    break
        return pd.Series([float(self.p(row)) for row in data.itertuples(index=False)], index=d.index)

In [63]:
# Function of preprocessing the data
def preprocessing(df):
    # FIND COLUMNS WITH MISSING VALUE
    missing_values_count = df.isnull().sum()/len(df)*100
    columns_with_missing_values = missing_values_count[missing_values_count > 0].index.tolist()
    columns_with_missing_values = list(set(columns_with_missing_values))

    # INPUT MISSING VALUE WITH KNNIMPUTER
    knn_imputer = KNNImputer(n_neighbors=5)
    df[columns_with_missing_values] = knn_imputer.fit_transform(df[columns_with_missing_values])

    #Drop the columns that are not useful
    x = df.drop(['ID', 'DAY_ID', 'FR_DE_EXCHANGE', 'FR_NET_EXPORT', 'DE_NET_EXPORT'], axis=1).fillna(df.mean(numeric_only=True))


    #Factorize the COUNTRY column
    x['COUNTRY'] = x['COUNTRY'].factorize()[0]

    # Create the new columns
    x['DE_NON_RENEWABLE'] = x['DE_GAS'] + x['DE_COAL'] + x['DE_LIGNITE'] + x['DE_NUCLEAR']
    x['DE_RENEWABLE'] = x['DE_HYDRO'] + x['DE_SOLAR'] + x['DE_WINDPOW']
    x['DE_EXCESS_ENERGY'] = x['DE_NON_RENEWABLE'] + x['DE_RENEWABLE'] - x['DE_CONSUMPTION']

    # Relation between the wind columns and the wind power column
    x['DE_WIND_SQCB'] = (x['DE_WIND'] - x['DE_WIND'].min()).pow(2.0/3.0)
    x['FR_WIND_SQCB'] = (x['FR_WIND'] - x['FR_WIND'].min()).pow(2.0/3.0)
    x['DE_WIND_CBRT'] = (x['DE_WIND'] - x['DE_WIND'].min()).pow(1.0/3.0)
    x['FR_WIND_CBRT'] = (x['FR_WIND'] - x['FR_WIND'].min()).pow(1.0/3.0)

    # Droping the columns that are not useful
    x = x[['DE_CONSUMPTION', 'FR_CONSUMPTION', 'DE_NET_IMPORT', 'FR_NET_IMPORT', 'DE_GAS', 'FR_GAS', 'DE_COAL',
       'FR_COAL', 'DE_HYDRO', 'FR_HYDRO', 'DE_NUCLEAR', 'FR_NUCLEAR',
       'DE_SOLAR', 'FR_SOLAR', 'DE_WINDPOW', 'FR_WINDPOW', 'DE_LIGNITE',
       'DE_RESIDUAL_LOAD', 'FR_RESIDUAL_LOAD', 'DE_RAIN', 'FR_RAIN', 'DE_WIND_SQCB',
       'FR_WIND_SQCB', 'DE_TEMP', 'FR_TEMP', 'GAS_RET', 'COAL_RET', 'CARBON_RET']]
    
    # Threshold values for cut-in speeds of German and French wind turbines, respectively.
    de_wind_switch = 1.5
    fr_wind_switch = 1.5

    # some post-split work to determine excess energy production
    # measuring excess wind energy production, determined by forecasted production based on wind levels
    wind_excess_lr = SDLinReg()
    wind_excess_lr.fit(x, 'DE_WIND_SQCB', 'DE_WINDPOW', lambda x, y : x > de_wind_switch)
    x['DE_WIND_EXCESS'] = wind_excess_lr.predict(x)

    wind_excess_lr.fit(x_train, 'FR_WIND_SQCB', 'FR_WINDPOW', lambda x, y : x > fr_wind_switch)
    x['FR_WIND_EXCESS'] = wind_excess_lr.predict(x)

   # french residual load is highly correlated with its consumption—seems to turn on/off non-renewable plants
   # in response to projected consumption, hence we detect overproduction by seeing if consumption is lower than the
   # prediction
    fr_consumption_lr = SDLinReg()
    fr_consumption_lr.fit(x_train, 'FR_RESIDUAL_LOAD', 'FR_CONSUMPTION')
    x['FR_OVERCON'] = fr_consumption_lr.predict(x)

   #Normalize the data
    norm_cols = ['GAS_RET', 'COAL_RET', 'CARBON_RET']
    x[norm_cols] = (x[norm_cols] - x[norm_cols].mean()) / x[norm_cols].std()

   #Create a new feature that clusters the DE_NUCLEAR column
    km = KMeans(n_clusters=6)
    km.fit(np.array(x['DE_NUCLEAR']).reshape(-1, 1))
    x['DE_NUCLEAR_CLUSTER'] = km.labels_

    # One hot encode the DE_NUCLEAR_CLUSTER column
    nuclear_onehot = onehot()
    nuclear_onehot.fit(np.array(x['DE_NUCLEAR_CLUSTER']).reshape(-1, 1))
    nuclear_onehot_cols = ['DE_NUCLEAR_0', 'DE_NUCLEAR_1', 'DE_NUCLEAR_2', 'DE_NUCLEAR_3', 'DE_NUCLEAR_4', 'DE_NUCLEAR_5']
    x[nuclear_onehot_cols] = pd.DataFrame(nuclear_onehot.transform(np.array(x['DE_NUCLEAR_CLUSTER']).reshape(-1, 1)).toarray(), index=x.index)

    return x 
    

In [64]:
x_train = preprocessing(df)
x_test = preprocessing(true_test)

/var/folders/dj/drqjc_c134xgs5ns31syhsm40000gn/T/ipykernel_52806/502714257.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return pd.Series([float(self.p(row)) for row in data.itertuples(index=False)], index=d.index)
/var/folders/dj/drqjc_c134xgs5ns31syhsm40000gn/T/ipykernel_52806/502714257.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return pd.Series([float(self.p(row)) for row in data.itertuples(index=False)], index=d.index)
/var/folders/dj/drqjc_c134xgs5ns31syhsm40000gn/T/ipykernel_52806/502714257.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you

In [66]:
# Save dataset
x_train.to_csv('data/final_dataset/x_train.csv', index=False)
x_test.to_csv('data/final_dataset/x_test.csv', index=False)